# IBM Model 1 with Expectation Maximization

## Author: Anmol Sharma

**Open the files**

In [1]:
from __future__ import print_function
import itertools, sys
import numpy as np

In [2]:
print('Opening files..', file=sys.stderr)
eng_set = open('../data/hansards.en', 'rb')
fr_set = open('../data/hansards.fr', 'rb')

Opening files..


**Perform Preprocessing**

1. **Split the data into two different sets, and split each sentence into words**
2. **Add a NONE character inside every english sentence**

In [3]:
src_sent = []
dest_sent = []

for line_en, line_fr in zip(eng_set, fr_set):
    # split each sentence into a list of words for easy processing
    src_sent.append(line_fr.split())
    dest_sent.append(line_en.split())

We can see the words contain many "\xc3\xa9es"... which are basically the unicode codes for special accented symbols in french. Nothing to worry. 

Also, the punctuation marks are left as it is as "words" which map directly to the punctuation in the destination language. 

In [4]:
print('French sentences..', file=sys.stderr)
print(src_sent[5:10], file=sys.stderr)

French sentences..
[['souvenons', '-', 'nous', ',', 'monsieur', 'le', 'Orateur', ',', 'que', 'ce', 'sont', 'ces', 'secteurs', 'de', 'notre', 'soci\xc3\xa9t\xc3\xa9', 'qui', 'servent', 'de', '\xc3\xa9pine', 'dorsale', '\xc3\xa0', 'notre', '\xc3\xa9conomie', '.'], ['bravo', '!'], ['bravo', '!'], ['monsieur', 'le', 'Orateur', ',', 'ma', 'question', 'se', 'adresse', '\xc3\xa0', 'le', 'ministre', 'charg\xc3\xa9', 'de', 'les', 'transports', '.'], ['tous', 'deux', 'poss\xc3\xa8dent', 'de', 'nombreuses', 'ann\xc3\xa9es', 'de', 'exp\xc3\xa9rience', 'dans', 'la', 'fabrication', 'et', 'la', 'distribution', 'de', 'les', 'produits', 'forestiers', '.']]


In [5]:
print('English sentences..', file=sys.stderr)
print(dest_sent[5:10], file=sys.stderr)

English sentences..
[['let', 'us', 'remember', ',', 'Mr.', 'Speaker', ',', 'that', 'these', 'segments', 'of', 'our', 'society', 'form', 'the', 'backbone', 'of', 'our', 'economy', '.'], ['hear', ',', 'hear', '!'], ['hear', ',', 'hear', '!'], ['Mr.', 'Speaker', ',', 'my', 'question', 'is', 'directed', 'to', 'the', 'Minister', 'of', 'Transport', '.'], ['both', 'have', 'many', 'years', 'experience', 'in', 'the', 'manufacture', 'and', 'distribution', 'of', 'forest', 'products', '.']]


## **We need to find the probability $t_k(f_i|e_j)$ where $f_i$ = french word (source language) and $e_j$ = english word (destination language)**

Find all the unique words in french data

In [6]:
# convert the source list into a chain of iterables, and then convert it to a set to only retain unique elements.
# further convert to list for easy processing
fr_vocab = list(set(itertools.chain.from_iterable(src_sent)))
en_vocab = list(set(itertools.chain.from_iterable(dest_sent)))

In [7]:
print('Some unique french words..', file=sys.stderr)
print(fr_vocab[0:5], file=sys.stderr)

Some unique french words..
['1,800', 'engendr\xc3\xa9e', 'encouragerais', 'accueille', 'accueilli']


In [8]:
print('Some unique english words..', file=sys.stderr)
print(en_vocab[0:5], file=sys.stderr)

Some unique english words..
['1,800', 'woods', 'hanging', 'trawling', 'originality']


# Start the training process..

**We cannot initialize the $t_k$ values to uniform due to memory constraints. A better way to do this is to first check if the key exists or not, and if it doesn't, then initialize it to uniform probability. This saves a huge memory and computational overhead of permuting through all $f_i$ and $e_j$ and setting them uniform, many of which will not even appear in the training text**

In [9]:
k = 0
t_k = {}
count_comb = {}
count_e = {}
uni_prob = 1.0 / np.shape(fr_vocab)[0]
epochs = 1

for _i in range(epochs):
    print('Currently on training epoch {}..'.format(_i+1), file=sys.stderr)
    # iterate over all training examples
    for src_sent_eg, dest_sent_eg in zip(src_sent, dest_sent):
        for f_i in src_sent_eg:
            Z = 0.0
            for e_j in dest_sent_eg:
                
                if (f_i, e_j) not in t_k:
#                     print('({}, {}) not in t_k, initializing to uniform!'.format(f_i, e_j))
                    t_k[(f_i, e_j)] = 1.0 / uni_prob
                
                Z += t_k[(f_i, e_j)]
            for e_j in dest_sent_eg:
                c = t_k[(f_i, e_j)] / Z
                
                # initialize counts on the fly
                if (f_i, e_j) not in count_comb:
#                     print('({}, {}) not in count_comb, initializing to 0!'.format(f_i, e_j))
                    count_comb[(f_i, e_j)] = 0
                
                # initialize counts on the fly
                if e_j not in count_e:
#                     print('({}) not in count_e, initializing to 0!'.format(e_j))
                    count_e[e_j] = 0
                    
                count_comb[(f_i, e_j)] += c
                count_e[e_j] += c
    print('Updating t_k counts...', file=sys.stderr)
    for f_e_keys in count_comb:
        # f_e_keys[0] = f_i, f_e_keys[1] = e_j
        t_k[(f_e_keys[0], f_e_keys[1])] = count_comb[f_e_keys] / count_e[f_e_keys[1]]

Currently on training epoch 1..
Updating t_k counts...


# Make predictions using this trained model..

In [11]:
print('Aligning...', file=sys.stderr)
print('Source | Destination', file=sys.stderr)
for src_sent_eg, dest_sent_eg in zip(src_sent, dest_sent):
    i = 0
    for f_i in src_sent_eg:
        bestp = 0
        bestj = 0
        j = 0
        for e_j in dest_sent_eg:
            if t_k[(f_i, e_j)] > bestp:
                bestp = t_k[(f_i, e_j)]
                bestj = e_j
                j += 1
        sys.stdout.write('{}-{} '.format(i,j))
        i += 1
    sys.stdout.write('\n')

Aligning...
Source | Destination


0-1 1-2 
0-1 1-2 
0-1 1-1 2-1 3-1 
0-1 1-1 2-1 3-1 
0-1 1-1 2-1 3-1 
0-3 1-1 2-2 3-4 4-4 5-3 6-3 7-4 8-4 9-4 10-5 11-4 12-4 13-8 14-4 15-5 16-4 17-4 18-8 19-3 20-3 21-4 22-4 23-4 24-2 
0-1 1-2 
0-1 1-2 
0-2 1-2 2-3 3-3 4-3 5-4 6-4 7-4 8-6 9-2 10-5 11-5 12-7 13-5 14-6 15-1 
0-3 1-1 2-4 3-3 4-3 5-4 6-3 7-5 8-5 9-4 10-3 11-3 12-4 13-4 14-3 15-4 16-5 17-4 18-3 
0-1 1-2 
0-3 1-4 2-2 3-5 4-5 5-6 6-4 7-5 8-6 9-3 10-5 11-4 12-5 13-4 14-4 15-7 16-4 17-3 
0-1 1-5 2-1 3-5 4-6 5-6 6-1 7-3 8-6 9-5 10-6 11-4 12-6 13-3 14-7 15-6 16-6 17-7 18-7 19-7 20-2 
0-4 1-5 2-6 3-2 4-3 5-4 6-4 7-5 8-7 9-4 10-4 11-5 12-3 13-6 14-6 15-4 16-3 
0-3 1-1 2-2 3-2 
0-1 1-2 
0-3 1-3 2-3 3-3 4-3 5-5 6-4 7-4 8-4 9-4 10-4 11-2 12-6 13-3 14-5 15-4 16-4 17-7 18-4 19-3 20-2 21-4 22-2 
0-3 1-3 2-4 3-3 4-3 5-3 6-4 
0-2 1-2 2-3 3-2 4-4 5-3 6-6 7-3 8-3 9-5 10-4 11-4 12-4 13-4 14-6 15-5 16-4 
0-5 1-4 2-4 3-3 4-3 5-4 6-3 7-5 8-4 9-5 10-3 11-3 12-3 13-4 14-3 15-4 16-3 17-3 18-3 19-3 20-3 21-3 22-4 23-6 
0-2 1-2 2-4 3-3 4-2 5-3 6-7 7-

0-1 1-2 2-4 3-4 4-6 5-3 6-2 7-6 8-7 9-5 10-4 11-6 12-3 13-4 14-6 15-5 16-5 17-5 18-3 19-2 20-6 21-6 22-8 23-8 24-4 
0-3 1-4 2-2 3-2 4-4 5-3 6-4 7-2 8-1 9-7 10-6 11-7 12-3 13-2 
0-2 1-2 2-4 3-3 4-3 5-5 6-3 7-5 8-7 9-4 10-8 11-5 12-3 13-7 14-3 15-4 16-5 17-4 18-6 19-2 20-5 21-4 22-4 23-1 
0-1 1-3 2-2 3-3 4-3 5-4 6-3 7-5 8-3 9-5 10-3 11-4 12-5 13-4 14-4 15-4 16-3 17-4 18-6 19-2 20-3 21-4 22-4 23-4 24-3 25-3 
0-3 1-1 2-2 3-1 4-1 5-2 6-2 7-1 8-3 9-4 10-2 11-2 12-3 13-3 14-1 15-3 16-3 17-2 18-3 
0-3 1-3 2-4 3-5 4-2 5-3 6-2 7-3 8-3 9-3 
0-5 1-3 2-2 3-6 4-2 5-6 6-5 7-3 8-6 9-3 10-5 11-6 12-6 13-3 14-6 15-3 16-6 17-5 18-4 
0-1 1-2 2-3 3-3 4-3 5-3 6-3 7-1 8-2 9-3 10-3 
0-3 1-3 2-2 3-1 4-3 5-4 6-4 7-4 8-5 9-3 10-2 
0-1 1-1 2-3 3-3 4-5 5-3 6-4 7-3 8-2 9-3 10-4 11-5 12-3 13-3 14-5 15-4 16-2 
0-3 1-3 2-3 3-4 4-4 5-4 6-4 7-4 8-4 9-3 10-4 11-4 
0-2 1-3 2-5 3-3 4-4 5-5 6-3 7-5 8-2 9-7 10-5 11-5 12-3 13-6 14-3 
0-4 1-3 2-2 3-2 4-4 5-3 6-4 7-4 8-4 9-4 10-6 11-4 12-4 13-4 14-3 15-5 16-7 17-5 
0-4 1-2 2-2 

0-1 1-3 2-2 3-3 4-3 5-2 6-5 7-5 8-4 9-9 10-3 11-5 12-6 13-5 14-9 15-5 16-8 17-4 18-3 19-3 20-5 21-5 22-5 23-2 24-5 25-2 
0-3 1-6 2-4 3-4 4-5 5-5 6-5 7-1 8-4 9-4 10-5 11-5 12-5 13-5 14-3 15-3 16-5 17-4 18-3 19-6 20-5 21-4 22-6 23-8 
0-2 1-5 2-4 3-3 4-5 5-5 6-4 7-3 8-2 9-3 10-3 11-3 12-4 
0-3 1-3 2-1 3-2 4-2 5-2 6-2 
0-2 1-3 2-3 3-3 4-5 5-6 6-3 7-3 8-3 9-4 10-3 11-7 12-7 13-6 14-3 15-5 16-3 17-6 18-5 19-6 20-6 21-4 22-5 23-4 24-4 25-6 
0-4 1-4 2-3 3-6 4-3 5-3 6-3 7-6 8-5 9-3 10-3 11-4 12-5 13-2 14-3 15-3 16-5 17-5 18-6 19-4 20-4 21-7 22-7 
0-1 1-3 2-2 3-6 4-7 5-6 6-5 7-8 8-5 9-4 10-6 11-6 12-6 13-7 14-4 15-7 16-8 17-7 18-8 19-7 20-6 21-2 
0-2 1-2 2-2 3-2 4-3 5-4 6-5 7-3 8-4 9-3 10-4 
0-2 1-3 2-2 3-4 4-4 
0-6 1-7 2-7 3-3 4-2 5-2 6-2 7-3 8-5 9-4 10-5 11-6 12-4 13-7 14-4 15-5 16-8 17-6 18-4 19-6 20-4 21-7 22-5 23-5 24-1 
0-2 1-3 2-5 3-4 4-5 5-3 6-5 7-5 8-3 9-3 10-3 11-4 12-3 13-4 14-4 15-3 16-5 
0-5 1-6 2-4 3-5 4-5 5-5 6-7 7-3 8-3 9-4 10-4 11-3 12-4 13-2 14-5 15-4 16-4 17-3 18-2 19-5 20-5 


0-1 1-2 2-2 3-2 4-2 5-2 6-2 7-4 8-4 
0-3 1-3 2-3 3-2 4-3 5-6 6-4 7-4 8-2 9-4 10-6 11-5 12-3 13-5 14-6 15-4 16-5 17-1 18-6 19-2 20-5 21-4 
0-3 1-3 2-3 3-1 4-3 5-2 6-4 
0-1 1-2 2-3 3-3 4-4 5-3 6-4 
0-2 1-2 2-3 3-2 4-3 5-3 6-4 7-5 8-3 9-5 10-4 11-5 12-4 13-5 14-4 15-5 
0-1 1-2 2-2 3-4 4-6 5-7 6-3 7-6 8-7 9-4 10-4 11-5 12-4 13-4 14-6 15-4 16-7 17-5 18-5 19-4 20-4 
0-1 1-2 2-4 3-3 4-7 5-3 6-5 7-2 8-3 9-7 10-6 11-6 12-2 13-4 14-4 15-3 16-4 17-7 18-3 19-5 20-7 21-3 22-3 23-7 24-4 25-7 26-7 27-3 28-3 
0-3 1-2 2-1 3-5 4-2 5-6 6-3 7-3 8-3 9-4 10-5 11-3 12-4 13-5 14-5 15-4 16-7 17-6 18-4 19-3 20-5 21-5 22-2 23-5 24-6 25-3 
0-5 1-5 2-5 3-4 4-2 5-3 6-8 7-7 8-4 9-2 10-4 11-4 12-4 13-4 14-5 15-8 16-4 17-4 18-5 19-5 20-5 21-5 22-4 23-3 
0-2 1-1 2-3 3-1 4-2 5-3 6-2 7-5 8-4 9-4 10-3 11-2 12-3 13-3 
0-2 1-2 2-2 3-2 4-2 5-2 6-2 7-2 8-2 9-2 10-2 11-2 12-2 13-2 14-2 15-3 16-2 17-5 18-4 19-2 20-5 21-2 22-2 23-2 24-6 
0-1 1-2 2-3 3-3 4-4 5-5 6-5 7-5 8-5 9-2 10-4 11-3 12-3 13-4 14-2 
0-5 1-4 2-4 3-3 4-6 5-6 6-

...

0-1 1-2 2-3 3-3 4-4 5-3 6-4 7-4 8-4 9-3 10-4 11-4 12-3 13-5 14-3 15-5 16-5 17-5 18-5 19-4 20-4 21-5 22-5 23-3 24-3 25-3 26-5 27-3 28-5 29-7 30-2 
0-3 1-4 2-4 3-7 4-8 5-4 6-5 7-6 8-7 9-3 10-4 11-5 12-6 13-6 14-3 15-5 16-4 17-4 18-4 19-3 20-3 21-3 22-4 23-7 24-5 25-4 26-8 27-5 28-7 29-5 30-2 31-4 32-7 33-6 34-6 35-7 36-8 37-7 38-7 39-9 40-4 41-5 42-6 43-6 44-5 45-5 46-3 
0-1 1-1 2-2 3-2 4-3 5-4 6-3 7-4 8-3 9-2 10-4 11-2 12-4 13-4 14-3 15-3 16-2 17-4 18-3 19-3 20-4 21-2 22-3 23-4 
0-1 1-1 2-1 
0-2 1-2 2-2 3-2 4-2 5-2 6-2 
0-3 1-3 2-3 3-3 4-3 5-4 6-5 7-4 8-5 9-4 10-5 11-5 12-4 13-4 14-3 15-4 16-6 
0-2 1-2 2-2 3-1 4-2 5-3 6-1 7-6 8-2 9-2 10-7 11-1 12-5 13-5 14-7 15-7 16-5 17-6 18-7 19-7 20-3 21-1 22-8 23-7 24-6 25-4 26-6 27-7 28-1 29-4 30-4 31-6 32-4 33-7 34-7 35-6 36-2 37-6 38-7 39-7 40-10 41-2 
0-1 1-4 2-2 3-5 4-5 5-5 6-4 7-5 8-5 9-4 10-5 11-5 12-1 13-4 14-2 15-5 16-5 17-5 18-5 19-5 20-4 21-5 22-5 23-5 24-6 25-5 26-5 27-4 28-5 29-4 30-5 31-6 32-5 33-5 34-3 35-5 36-5 37-4 38-3 
0-2 1-2 2-2

0-1 1-4 2-2 3-4 4-3 5-6 6-4 7-4 8-5 9-7 10-8 11-6 12-5 13-5 14-3 15-6 16-4 17-3 18-3 19-3 20-3 21-7 22-6 23-5 24-3 25-4 26-6 27-5 28-4 29-8 30-5 31-3 32-7 33-7 34-6 35-5 36-7 37-8 38-7 39-10 40-7 41-7 42-7 43-8 44-2 
0-2 1-2 2-3 3-4 4-3 5-4 6-3 7-3 8-3 9-5 10-3 11-4 12-5 13-6 
0-2 1-2 2-2 3-3 4-4 5-4 6-4 7-5 8-5 9-5 10-4 11-4 12-1 
0-3 1-2 2-2 3-5 4-2 5-2 6-2 7-2 8-2 9-2 10-2 11-3 12-3 13-1 14-2 15-2 16-3 17-2 
0-2 1-2 2-3 3-3 4-4 5-3 6-4 7-3 8-4 9-1 10-3 11-3 12-4 13-5 
0-1 1-3 2-3 3-3 4-3 5-2 6-3 7-4 8-3 9-3 10-4 
0-4 1-3 2-3 3-5 4-4 5-3 6-4 7-5 8-5 9-4 10-3 11-7 12-7 13-7 14-5 
0-4 1-2 2-4 3-3 4-3 5-3 6-4 7-4 8-4 9-3 10-3 11-3 12-2 13-3 14-4 15-4 16-2 17-3 18-2 19-3 20-3 
0-3 1-3 2-3 3-5 4-4 5-5 6-4 7-6 8-4 9-5 10-4 
0-2 1-2 2-2 3-3 4-2 5-3 6-6 7-7 8-4 9-6 10-7 11-8 12-2 13-5 14-8 15-9 16-8 17-1 
0-1 1-3 2-4 3-4 4-3 5-4 6-5 7-3 8-4 9-4 10-3 11-2 12-4 13-5 14-3 15-2 16-5 17-3 18-2 19-5 20-7 21-5 22-2 23-5 24-4 25-5 26-2 27-4 28-4 29-6 30-2 31-5 32-6 33-7 34-6 35-5 36-3 37-2 38-2 39-8

0-2 1-2 2-2 3-3 4-6 5-5 6-6 7-6 8-4 9-7 10-4 11-3 12-6 13-3 14-6 15-5 16-2 17-4 18-4 19-6 20-3 21-5 22-7 23-6 24-6 25-6 26-6 27-7 28-8 29-5 30-5 31-5 32-4 33-7 34-7 35-6 36-6 37-7 38-5 39-7 40-7 41-5 42-4 43-5 44-6 45-8 46-6 47-2 48-6 49-6 50-7 51-8 
0-1 1-2 2-3 3-4 4-7 5-3 6-6 7-3 8-4 9-2 10-4 11-7 12-6 13-3 14-2 15-3 16-4 17-7 18-6 19-3 20-4 21-6 22-5 23-6 24-5 25-3 26-6 27-5 28-5 29-4 30-2 31-4 32-6 33-6 34-7 35-9 36-6 37-4 38-4 
0-2 1-2 2-4 3-3 4-4 5-3 6-3 7-6 8-3 9-5 10-5 11-7 
0-2 1-2 2-2 3-3 4-5 5-3 6-4 7-5 8-6 9-5 10-7 11-1 
0-4 1-2 2-4 3-4 4-5 5-2 6-2 7-3 8-3 9-4 10-5 11-5 12-5 13-4 14-6 15-5 16-4 17-2 18-5 19-5 20-4 21-5 22-3 23-5 24-4 25-5 26-4 27-2 28-5 29-5 30-2 31-5 32-4 33-4 34-6 35-4 
0-2 1-4 2-2 3-4 4-5 5-5 6-3 7-3 8-4 9-4 10-6 11-3 12-5 13-4 14-3 15-3 16-7 17-7 18-7 19-5 20-4 21-6 22-5 23-5 24-7 25-5 26-4 27-6 
0-2 1-2 2-4 3-3 4-3 5-3 6-4 7-2 8-2 9-6 10-3 11-6 12-3 13-3 14-3 15-4 16-4 17-4 18-6 19-3 20-4 21-4 22-4 23-7 24-6 25-3 
0-1 1-1 2-1 
0-2 1-2 2-2 
0-2 1-2 2-2 

0-2 1-3 2-2 3-3 4-3 5-4 6-4 7-4 8-5 9-3 10-5 11-7 12-3 13-6 14-1 
0-1 1-2 2-5 3-3 4-3 5-5 6-3 7-3 8-4 9-5 10-3 11-3 12-4 13-7 14-2 15-4 16-6 17-4 18-5 19-5 20-6 21-5 22-7 23-6 24-5 25-6 26-5 27-6 28-5 29-5 30-7 31-5 32-3 33-6 34-5 35-6 36-5 37-8 38-4 39-8 40-7 41-5 42-6 43-6 44-5 45-7 46-6 47-6 48-5 
0-3 1-3 2-3 3-5 4-5 5-4 6-5 7-3 8-4 9-1 10-7 11-5 12-3 13-5 14-4 15-5 16-5 17-3 18-5 19-5 20-6 21-5 22-5 23-6 24-8 25-4 
0-2 1-2 2-3 
0-2 1-4 2-3 3-3 4-2 5-3 6-3 7-6 
0-2 1-2 2-2 3-2 4-2 5-4 6-3 7-2 8-3 9-4 10-2 11-3 12-3 13-4 14-5 15-2 16-4 17-5 
0-2 1-2 2-2 3-3 4-3 5-5 6-6 7-4 8-3 9-8 10-2 11-5 12-9 13-7 14-8 15-7 16-3 17-11 18-4 19-2 20-8 21-1 
0-1 1-1 2-2 3-3 4-4 5-4 6-2 7-3 8-2 9-3 10-4 11-4 12-3 
0-1 1-2 2-2 3-3 4-5 5-4 6-4 7-4 8-4 9-3 10-5 11-2 12-2 13-3 14-5 15-6 16-6 17-6 18-5 19-4 
0-1 1-1 2-1 
0-1 1-1 
0-1 1-3 2-3 3-3 4-3 5-2 6-3 7-3 8-3 9-3 10-3 11-4 12-4 13-3 14-2 
0-2 1-2 2-2 3-3 4-8 5-7 6-6 7-4 8-5 9-5 10-3 11-2 12-3 13-10 14-4 15-5 16-6 17-6 18-6 19-8 20-6 21-9 22-11 23-8 2

KeyboardInterrupt: 